In [1]:
pip install yfinance numpy scipy


Defaulting to user installation because normal site-packages is not writeableNote: you may need to restart the kernel to use updated packages.



In [9]:
import numpy as np
import pandas as pd
import yfinance as yf
from scipy.stats import norm

# Black-Scholes Put Option Pricing Function
def black_scholes_put(S, K, T, r, sigma):
    """
    Calculates the Black-Scholes price of a European put option.
    """
    d1 = (np.log(S / K) + (r + 0.5 * sigma**2) * T) / (sigma * np.sqrt(T))
    d2 = d1 - sigma * np.sqrt(T)
    put_price = K * np.exp(-r * T) * norm.cdf(-d2) - S * norm.cdf(-d1)
    return put_price

# Vega of the Option (Sensitivity to Volatility)
def vega(S, K, T, r, sigma):
    """
    Calculates Vega (derivative of option price with respect to volatility).
    """
    d1 = (np.log(S / K) + (r + 0.5 * sigma**2) * T) / (sigma * np.sqrt(T))
    return S * norm.pdf(d1) * np.sqrt(T)

# Bisection Method for Implied Volatility
def bisection_iv(S, K, T, r, market_price, tol=1e-5, max_iter=100):
    """
    Fallback method: Bisection method for implied volatility.
    """
    low, high = 0.01, 3.0  # Bounds for volatility
    for _ in range(max_iter):
        mid = (low + high) / 2.0
        price = black_scholes_put(S, K, T, r, mid)
        if abs(price - market_price) < tol:
            return mid
        if price > market_price:
            high = mid
        else:
            low = mid
    return mid  # Return approximate solution

# Newton-Raphson Method with Fallback to Bisection
def newton_raphson_iv(S, K, T, r, market_price, tol=1e-5, max_iter=100):
    """
    Newton-Raphson method to estimate implied volatility.
    Falls back to Bisection method if convergence is unlikely.
    """
    sigma = 0.5  # Initial guess
    for _ in range(max_iter):
        price = black_scholes_put(S, K, T, r, sigma)
        diff = price - market_price
        if abs(diff) < tol:
            return sigma
        
        vega_value = vega(S, K, T, r, sigma)
        if vega_value < 1e-5:  # Vega too small, fallback to Bisection
            print("Switching to Bisection method due to small Vega.")
            return bisection_iv(S, K, T, r, market_price, tol, max_iter)
        
        sigma -= diff / vega_value
        sigma = max(0.01, min(sigma, 3.0))  # Restrict sigma to a reasonable range
    
    print("Switching to Bisection method due to Newton-Raphson failure.")
    return bisection_iv(S, K, T, r, market_price, tol, max_iter)

# Main Code to Fetch Data and Estimate Implied Volatility
def main():
    # Fetch Tesla's Spot Price and Put Option Prices
    ticker = "TSLA"
    stock = yf.Ticker(ticker)
    spot_price = stock.history(period="1d").iloc[-1]["Close"]  # Spot Price
    print(f"Spot Price of Tesla (TSLA): {spot_price:.2f}")

    # Assumptions
    expiry_date = "2025-01-17"
    current_date = pd.Timestamp.now()
    T = (pd.to_datetime(expiry_date) - current_date).days / 365.0  # Time to Maturity in Years
    risk_free_rate = 0.05  # Risk-free rate (5%)

    # Strike Prices and Option Market Prices (Assumed Values Here)
    strike_prices = [140, 340]  # K=140 and K=340
    put_prices = [30.0, 5.0]  # Example Market Put Prices (Modify with real data if available)

    print("\nEstimating Implied Volatility:")
    for i, K in enumerate(strike_prices):
        market_price = put_prices[i]
        try:
            implied_vol = newton_raphson_iv(spot_price, K, T, risk_free_rate, market_price)
            print(f"Implied Volatility for Strike Price K={K}: {implied_vol:.4f}")
        except ValueError as e:
            print(f"Error for Strike Price K={K}: {e}")

if __name__ == "__main__":
    main()


Spot Price of Tesla (TSLA): 424.77

Estimating Implied Volatility:
Switching to Bisection method due to small Vega.
Implied Volatility for Strike Price K=140: 3.0000
Implied Volatility for Strike Price K=340: 0.6533


In [11]:
import numpy as np
import pandas as pd
import yfinance as yf
from scipy.stats import norm

# Black-Scholes Put Option Pricing Function
def black_scholes_put(S, K, T, r, sigma):
    """
    Calculates the Black-Scholes price of a European put option.
    """
    d1 = (np.log(S / K) + (r + 0.5 * sigma**2) * T) / (sigma * np.sqrt(T))
    d2 = d1 - sigma * np.sqrt(T)
    put_price = K * np.exp(-r * T) * norm.cdf(-d2) - S * norm.cdf(-d1)
    return put_price

# Vega of the Option (Sensitivity to Volatility)
def vega(S, K, T, r, sigma):
    """
    Calculates Vega (derivative of option price with respect to volatility).
    """
    d1 = (np.log(S / K) + (r + 0.5 * sigma**2) * T) / (sigma * np.sqrt(T))
    return S * norm.pdf(d1) * np.sqrt(T)

# Bisection Method for Implied Volatility
def bisection_iv(S, K, T, r, market_price, tol=1e-5, max_iter=100):
    """
    Fallback method: Bisection method for implied volatility.
    """
    low, high = 0.01, 3.0  # Bounds for volatility
    for _ in range(max_iter):
        mid = (low + high) / 2.0
        price = black_scholes_put(S, K, T, r, mid)
        if abs(price - market_price) < tol:
            return mid
        if price > market_price:
            high = mid
        else:
            low = mid
    return mid  # Return approximate solution

# Newton-Raphson Method with Fallback to Bisection
def newton_raphson_iv(S, K, T, r, market_price, tol=1e-5, max_iter=100):
    """
    Newton-Raphson method to estimate implied volatility.
    Falls back to Bisection method if convergence is unlikely.
    """
    sigma = 0.5  # Initial guess
    for _ in range(max_iter):
        price = black_scholes_put(S, K, T, r, sigma)
        diff = price - market_price
        if abs(diff) < tol:
            return sigma
        
        vega_value = vega(S, K, T, r, sigma)
        if vega_value < 1e-5:  # Vega too small, fallback to Bisection
            print("Switching to Bisection method due to small Vega.")
            return bisection_iv(S, K, T, r, market_price, tol, max_iter)
        
        sigma -= diff / vega_value
        sigma = max(0.01, min(sigma, 3.0))  # Restrict sigma to a reasonable range
    
    print("Switching to Bisection method due to Newton-Raphson failure.")
    return bisection_iv(S, K, T, r, market_price, tol, max_iter)

# Fetch Spot Price and Put Option Prices from Yahoo Finance
def fetch_option_data(ticker, expiry_date, strike_prices):
    """
    Fetch spot price and put option prices for given strikes and expiry date.
    """
    stock = yf.Ticker(ticker)

    # Fetch current stock price
    spot_price = stock.history(period="1d").iloc[-1]["Close"]
    print(f"Spot Price of {ticker}: ${spot_price:.2f}")

    # Fetch option chain data for the given expiry date
    option_chain = stock.option_chain(expiry_date)
    puts = option_chain.puts  # Get all put options

    # Extract put prices for the desired strike prices
    put_prices = []
    for strike in strike_prices:
        put_price = puts.loc[puts['strike'] == strike, 'lastPrice']
        if not put_price.empty:
            put_prices.append(put_price.values[0])
        else:
            put_prices.append(None)  # Handle missing data

    print("\nPut Option Prices:")
    for i, strike in enumerate(strike_prices):
        price = put_prices[i]
        print(f"Strike Price: {strike}, Put Price: {price if price else 'N/A'}")

    return spot_price, put_prices

# Main Code to Fetch Data and Estimate Implied Volatility
def main():
    # Ticker and expiry date
    ticker = "TSLA"
    expiry_date = "2025-01-17"
    strike_prices = [140, 340]  # Strike prices

    # Risk-free rate
    risk_free_rate = 0.05

    # Time to maturity
    current_date = pd.Timestamp.now()
    T = (pd.to_datetime(expiry_date) - current_date).days / 365.0

    # Fetch data
    spot_price, put_prices = fetch_option_data(ticker, expiry_date, strike_prices)

    # Estimate Implied Volatility
    print("\nEstimating Implied Volatility:")
    for i, K in enumerate(strike_prices):
        market_price = put_prices[i]
        if market_price is not None:
            try:
                implied_vol = newton_raphson_iv(spot_price, K, T, risk_free_rate, market_price)
                print(f"Implied Volatility for Strike Price K={K}: {implied_vol:.4f}")
            except ValueError as e:
                print(f"Error for Strike Price K={K}: {e}")
        else:
            print(f"Implied Volatility for Strike Price K={K}: N/A (Price not available)")

if __name__ == "__main__":
    main()


Spot Price of TSLA: $424.77

Put Option Prices:
Strike Price: 140, Put Price: 0.18
Strike Price: 340, Put Price: 4.9

Estimating Implied Volatility:
Switching to Bisection method due to small Vega.
Implied Volatility for Strike Price K=140: 1.4128
Implied Volatility for Strike Price K=340: 0.6492
